## Model Building for Movie Data

This part mainly focuses on building models to predict movie revenue for the test data. We will use linear regression model as a baseline and then try tree-based models.

In [1]:
# Make the chunk output all results not only the last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
import datetime
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
import os
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
# from catboost import CatBoostRegressor
# import eli5
# import shap

/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [11]:
# Load data
train=pd.read_csv("new_train_2.csv")
test=pd.read_csv("new_test_2.csv")

# Look at the data
train.head(10)

# Shape
train.shape
test.shape

,id,budget,original_language,original_title,overview,popularity,runtime,status,tagline,title,Keywords,cast,crew,revenue,collection,has_collection,log_budget,genres_name,n_genres,all_genres,genre_Drama,genre_Comedy,genre_Thriller,genre_Action,genre_Romance,genre_Crime,genre_Adventure,genre_Horror,genre_Science Fiction,genre_Family,genre_Fantasy,has_homepage,log_popularity,company,n_company,all_production_companies,production_company_Warner Bros.,production_company_Universal Pictures,production_company_Paramount Pictures,production_company_Twentieth Century Fox Film Corporation,production_company_Columbia Pictures,production_company_Metro-Goldwyn-Mayer (MGM),production_company_New Line Cinema,production_company_Touchstone Pictures,production_company_Walt Disney Pictures,production_company_Columbia Pictures Corporation,production_company_TriStar Pictures,production_company_Relativity Media,production_company_Canal+,production_company_United Artists,production_company_Miramax Films,production_company_Village Roadshow Pictures,production_company_Regency Enterprises,production_company_BBC Films,production_company_Dune Entertainment,production_company_Working Title Films,country,n_country,all_production_countries,production_country_United States of America,production_country_United Kingdom,production_country_France,production_country_Germany,production_country_Canada,production_country_India,production_country_Italy,production_country_Japan,production_country_Australia,production_country_Russia,production_country_Spain,production_country_China,production_country_Hong Kong,production_country_Ireland,production_country_Belgium,release_time,year,month,DOW,season,Weekend,language,n_lan,all_spoken_languages,lan_English,lan_Français,lan_Español,lan_Deutsch,lan_Pусский,lan_Italiano,lan_日本語,lan_普通话,word,word_div,Crew,n_crew,initial_crew,crew_top,crew_top_ratio,Cast,n_cast,initial_cast,cast_top,cast_top_ratio,log_revenue
0,1,14000000,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,93.0,Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,Hot Tub Time Machine Collection,1,16.454568,['Comedy'],1,Comedy,0,1,0,0,0,0,0,0,0,0,0,0,2.024905,"['Paramount Pictures', 'United Artists', 'Metr...",3,Metro-Goldwyn-Mayer (MGM) Paramount Pictures U...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,['United States of America'],1,United States of America,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015-02-20,2015,2,4,4,0,['English'],1,English,1,0,0,0,0,0,0,0,"['time travel', 'sequel', 'hot tub', 'duringcr...",4,"['Kelly Cantley', 'Steve Pink', 'Josh Heald', ...",72,"['Kelly Cantley', 'Steve Pink', 'Josh Heald', ...",1,0.013889,"['Rob Corddry', 'Craig Robinson', 'Clark Duke'...",24,"['Rob Corddry', 'Craig Robinson', 'Clark Duke'...",3,0.125000,16.326300
1,2,40000000,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,113.0,Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,The Princess Diaries Collection,1,17.504390,"['Comedy', 'Drama', 'Family', 'Romance']",4,Comedy Drama Family Romance,1,1,0,0,1,0,0,0,0,1,0,0,2.224504,['Walt Disney Pictures'],1,Walt Disney Pictures,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,['United States of America'],1,United States of America,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2004-08-06,2004,8,4,2,0,['English'],1,English,1,0,0,0,0,0,0,0,"['coronation', 'duty', 'marriage', 'falling in...",4,"['Garry Marshall', 'Charles Minsky', 'John Deb...",9,"['Garry Marshall', 'Charles Minsky', 'John Deb...",2,0.222222,"['Anne Hathaway', 'Julie Andrews', 'H√©ctor El...",20,"['Anne Hathaway', '

(3000, 104)

(4398, 102)

In [12]:
list(train.columns)

['id',
 'budget',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'runtime',
 'status',
 'tagline',
 'title',
 'Keywords',
 'cast',
 'crew',
 'revenue',
 'collection',
 'has_collection',
 'log_budget',
 'genres_name',
 'n_genres',
 'all_genres',
 'genre_Drama',
 'genre_Comedy',
 'genre_Thriller',
 'genre_Action',
 'genre_Romance',
 'genre_Crime',
 'genre_Adventure',
 'genre_Horror',
 'genre_Science Fiction',
 'genre_Family',
 'genre_Fantasy',
 'has_homepage',
 'log_popularity',
 'company',
 'n_company',
 'all_production_companies',
 'production_company_Warner Bros.',
 'production_company_Universal Pictures',
 'production_company_Paramount Pictures',
 'production_company_Twentieth Century Fox Film Corporation',
 'production_company_Columbia Pictures',
 'production_company_Metro-Goldwyn-Mayer (MGM)',
 'production_company_New Line Cinema',
 'production_company_Touchstone Pictures',
 'production_company_Walt Disney Pictures',
 'production_company_Columbia Pictures Cor

In [21]:
X_train=train.drop(['original_language','original_title','overview','status','tagline',
                    'title','Keywords','cast','crew','collection','log_budget','genres_name',
                    'all_genres','log_popularity','company','all_production_companies','country',
                    'all_production_countries','release_time','language','all_spoken_languages',
                    'word','Crew','initial_crew','crew_top','Cast','initial_cast','cast_top',
                    'log_revenue','revenue'], axis=1)
Y_train=train.revenue
X_test=test.drop(['original_language','original_title','overview','status','tagline',
                    'title','Keywords','cast','crew','collection','log_budget','genres_name',
                    'all_genres','log_popularity','company','all_production_companies','country',
                    'all_production_countries','release_time','language','all_spoken_languages',
                    'word','Crew','initial_crew','crew_top','Cast','initial_cast','cast_top',], axis=1)

In [ ]:
# Fix previous data problem

In [22]:
# Split the train data
# Since the train data does not have many observations, we split the train and valid dataset 
# as 90% and 10% separately
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.1)

### Linear Regression Model

In [ ]:
# Build a linear regression model as a baseine
linear_model=LinearRegression()
linear_model.fit(X_train, Y_train)

### Tree-Based Model

In [16]:
# The first model is built using lightgbm
params = {'num_leaves': 30,
         'min_data_in_leaf': 20,
         'objective': 'regression',
         'max_depth': 5,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2,
         "verbosity": -1}
model1 = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
model1.fit(X_train, Y_train, 
           eval_set=[(X_train, Y_train), (X_valid, Y_valid)], eval_metric='rmse',
           verbose=1000, early_stopping_rounds=200)

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[547]	training's rmse: 5.42874e+07	valid_1's rmse: 7.68631e+07


LGBMRegressor(bagging_fraction=0.9, bagging_freq=1, bagging_seed=11,
       boosting='gbdt', boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, feature_fraction=0.9, importance_type='split',
       lambda_l1=0.2, learning_rate=0.01, max_depth=5, metric='rmse',
       min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=20,
       min_split_gain=0.0, n_estimators=20000, n_jobs=-1, nthread=4,
       num_leaves=30, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0, verbosity=-1)

In [ ]:
# Let's look at how much each predictor can explain
explainer = shap.TreeExplainer(model1, X_train)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values, X_train)